In [115]:
%matplotlib widget
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style(style='ticks')
import matplotlib.pyplot as plt
import seaborn as sns

from lab.classes.dbclasses import dbMouse, dbExperiment
from lab.classes import ExperimentGroup
# import lab.analysis.imaging_analysis as ia

import numpy as np
import pandas as pd

import sys
sys.path.insert(0, '/home/sebi/code/analysis/analysis-scripts/induction_analysis')
from Experiments.IOExperiment import IOExperiment
import Analysis.induction_analysis as ia

/home/sebi/code/analysis/analysis-scripts/induction_analysis/Analysis/induction_analysis.py:4: UserWarning: 
This call to matplotlib.use() has no effect because the backend has already
been chosen; matplotlib.use() must be called *before* pylab, matplotlib.pyplot,
or matplotlib.backends is imported for the first time.

The backend was *originally* set to 'module://ipympl.backend_nbagg' by the following code:
  File "/usr/lib/python2.7/runpy.py", line 174, in _run_module_as_main
    "__main__", fname, loader, pkg_name)
  File "/usr/lib/python2.7/runpy.py", line 72, in _run_code
    exec code in run_globals
  File "/home/sebi/.local/lib/python2.7/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/home/sebi/.local/lib/python2.7/site-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/home/sebi/.local/lib/python2.7/site-packages/ipykernel/kernelapp.py", line 499, in start
    self.io_loop.start()
  

In [88]:
def formatPlot(ax, offset=10):
    for axis in ['left', 'right', 'bottom', 'top']:
        ax.spines[axis].set_linewidth(2)
    ax.tick_params(labelsize=14, width=2, length=4)
    sns.despine(ax=ax, offset=offset)
    ax.set_xlabel('')
    ax.set_ylabel('')

In [24]:
mice = ['msa0919_1', 'msa0919_2', 'msa0919_3', 'msa1119_4']
expts = []
for mouse in mice:
    expts.extend(dbMouse(mouse).findall('experiment'))

In [36]:
grp  = []

stim_file = '/home/sebi/code/analysis/analysis-scripts/induction_analysis/io_mohsin_experiments.csv'
with open(stim_file, 'rb') as f:
    for line in f:
        exp, mark, e_type = line.split(',')
        exp = exp.strip(' ')
        mark = mark.strip(' ')
        
        eid = [x.trial_id for x in expts if exp == x.get('tSeriesDirectory')][0]

        grp.append(IOExperiment(eid, mark))

In [38]:
expts = sorted(grp, key=lambda x: x.attrib['LED_setting'])

In [42]:
stims = {}
for expt in expts:
    try:
        stims[expt.totuple()] = (expt.behaviorData()['led_context_pin'][:, 0] / expt.frame_period()).astype(int)
    except KeyError:
        print expt.trial_id

In [181]:
def psth(sig, stim_idx, pre, post, stim_dur):
    roi_psth = []
    mean = np.nanmean(sig)
    std = np.nanstd(sig)
    zsig = (sig - mean) / std
    for stim_frame in stim_idx:
        if stim_frame == 0:
            continue
        roi_psth.append(zsig[stim_frame - pre:stim_frame + stim_dur + post + 1])
    
    return np.nanmean(roi_psth, axis = 0)

In [177]:
def stim_response(sig, stim_idx, pre, post, stim_dur):
    roi_psth = []
    mean = np.nanmean(sig)
    std = np.nanstd(sig)
    zsig = (sig - mean) / std
    for stim_frame in stim_idx:
        # TODO are cells actually being stimmed at 0?
        if stim_frame == 0:
            continue
#         roi_psth.append(np.hstack([sig[stim_frame - pre:stim_frame],
#                                    sig[stim_frame + stim_dur:stim_frame + stim_dur + post + 1]]))
        roi_psth.append(np.nanmean(zsig[stim_frame + stim_dur:stim_frame + stim_dur + post + 1]) -
                        np.nanmean(zsig[stim_frame - pre:stim_frame]))
    
    return np.nanmean(roi_psth)
    

In [102]:
idx = expt.find_targeted_cells()

In [103]:
def find_targeted_cells(expt, label='s2p'):
    expt._time_average = expt.imaging_dataset().time_averages[0, ..., -1]

    rois = self.rois(label=label)
#     centroids = [self._com(roi) for roi in rois]
    dists = [roi.polygons[0].distance()]

    idx = [self._closest_cell(centroids, point) for point in self.stim_locations]
    return idx

[32, 10, 14]

In [112]:
from shapely.geometry import Point
from sima.ROI import mask2poly

def dist_to_target(grp, stim_filter=None):
    # Distance of stimmed cells to target
    # Everything done in pixel space until final conversion
    cell_names = ['Cell_A', 'Cell_B', 'Cell_C']
    dists = []
    for expt in grp:
        rois = expt.rois()
        expt._time_average = expt.imaging_dataset().time_averages[0, ..., -1]
        spiral_height = expt.spiral_sizes[0] * expt.frame_shape()[2]
        
        mouse_name = expt.parent.mouse_name
        eid = expt.trial_id
        
        for stim_name, stim_loc in zip(cell_names, expt.stim_locations):

            spiral_center = Point(stim_loc)
            spiral_footprint = spiral_center.buffer(spiral_height / 2.)

            for roi in rois:
                # TODO why do we have to transpose here??
                roi_poly = mask2poly(ia.binarize_mask(roi).T)

                dist = roi_poly.distance(spiral_footprint) * ia.get_microns(expt)[0]

                dists.append({'mouse_name': mouse_name,
                              'expt_id': eid,
                              'cell': stim_name,
                              'roi': roi.label,
                              'dist': dist})

    return pd.DataFrame(dists)

In [191]:
grp[-9].LED_setting

60

In [193]:
roi_labels

['0000-0077-0365-4080599747994258788',
 '0000-0066-0345-4080599747994258788',
 '0000-0330-0095-4080599747994258788',
 '0000-0327-0409-4080599747994258788',
 '0000-0343-0425-4080599747994258788',
 '0000-0314-0418-4080599747994258788']

In [195]:
expt.imagingData(dFOverF='from_file', roi_filter=roi_filter).shape

(0, 5356, 1)

In [201]:
rois = expt.rois(roi_filter=roi_filter)

In [202]:
for r in rois:
    print r.tags

set(['stimmed_Cell_A', 'Cell_A'])
set([])
set(['stimmed_Cell_C', 'Cell_C'])
set([])
set([])
set(['Cell_B'])


In [200]:
fig = plt.figure()
ax = fig.add_subplot(111)

expt = grp[-9]
roi_labels = cell_dists.loc[(cell_dists['expt_id'] == expt.trial_id) & (cell_dists['dist'] == 0)]['roi'].tolist()
roi_filter = lambda x: x.label in roi_labels

sigs = expt.imagingData(dFOverF='from_file', roi_filter=roi_filter)

sigs = (sigs - np.nanmean(sigs, axis=1, keepdims=True)) / np.nanstd(sigs, axis=1, keepdims=True)

for i, sig in enumerate(sigs):
    
    ax.plot(sig + 6*i)

stims = (expt.behaviorData()['led_context_pin'][:, 0] / expt.frame_period()).astype(int)
for stim in stims:
    ax.axvline(stim, color='k')

FigureCanvasNbAgg()

In [326]:
data_list = []

cell_names = ['Cell_A', 'Cell_B', 'Cell_C']
# roi_filter = lambda x: any([cell_name in x.tags for cell_name in ['Cell_A', 'Cell_B', 'Cell_C']])

pre_sec = 1
post_sec = 0.5
stim_sec = 0.5

for expt in expts:
    
    mouse_name = expt.parent.mouse_name
    
    if expt.get('LED_setting') in [70, 80]:
        continue
    
    stim_dur = int(stim_sec / expt.frame_period())
    pre = int(pre_sec / expt.frame_period())
    post = int(post_sec / expt.frame_period())
    stims = (expt.behaviorData()['led_context_pin'][:, 0] / expt.frame_period()).astype(int)
    
    for cell in xrange(3):
        cell_stims = stims[cell::3]
        roi_filter = None
#         roi_filter = lambda x: cell_names[cell] in x.tags
#         adjacent_labels = cell_dists.loc[(cell_dists['expt_id'] == expt.trial_id) & (cell_dists['cell'] == cell_names[cell]) & (cell_dists['dist'] == 0)]['roi'].tolist()
    
#         if len(adjacent_labels) == 0:
#             print expt.get('tSeriesDirectory'), cell_names[cell]
#             continue
#         roi_filter = lambda x: x.label in adjacent_labels
        
        
        sigs = expt.imagingData(dFOverF='from_file', roi_filter=roi_filter)[:, :, 0]
        rois = expt.rois(roi_filter=roi_filter)
        for sig, roi in zip(sigs, rois):
            
#             if roi.label not in adjacent_labels:
#                 continue
            rpsth = psth(sig, cell_stims, pre, 2 * post, stim_dur)
            for stim_frame in stims:
                sig[stim_frame:stim_frame + stim_dur + 1] = np.nan

#             response = stim_response(sig, cell_stims, pre, post, stim_dur)
            response = stim_response(sig, cell_stims, pre, post, stim_dur)

            data_list.append({'expt_id': expt.trial_id,
                              'mouse_name': mouse_name,
                              'value': response,
                              'LED': int(expt.get('LED_setting')),
                              'cell': cell_names[cell],
                              'roi': roi.label,
                              'target': cell_names[cell] in roi.tags,
                              'stim': any([x.startswith('stimmed') for x in roi.tags]),
                              'psth': rpsth})
        
df = pd.DataFrame(data_list)
df['mouse_cell'] = df['mouse_name'] + df['cell']

In [345]:
fig = plt.figure()
ax = fig.add_subplot(111)
plot_df = df.loc[df['target']]


# sns.lineplot(y='value', x='LED', hue='mouse_cell', data=df, alpha=0.5, color='0.5', ax=ax)
# group_df = df.groupby(['LED', 'mouse_cell'], as_index=False).mean()
g = sns.pointplot(y='value', x='LED', data=plot_df, ci='sd', color='k', ax=ax)
for l in g.lines:
    plt.setp(l,linewidth=1)

sns.swarmplot(y='value', x='LED', data=plot_df, color='0.5')

pfit = np.polyfit(x=plot_df['LED'].values[:], y=plot_df['value'].values[:], deg=1)
x = np.linspace(plot_df['LED'].min(), plot_df['LED'].max(), 2)
y = x * pfit[0] + pfit[1]
ax.plot([0, 6], y, color='k', zorder=0, ls='--')


formatPlot(ax)
ax.set_xlabel('Laser Power', fontsize=18)
ax.set_ylabel(r'Stim Response ($\sigma$)', fontsize=18)

plt.tight_layout()

fig.savefig('/home/sebi/plots/ind/figs3/power_vs_response_all.svg', bbox_inches='tight')

FigureCanvasNbAgg()

In [330]:
merge_df = df.merge(cell_dists, on=['expt_id', 'mouse_name', 'roi', 'cell'])

In [230]:
from scipy.stats import binned_statistic, sem

In [341]:
fig = plt.figure()
ax = fig.add_subplot(111)

target_df = df.loc[df['target']]

colors = sns.cubehelix_palette(7)
led_settings = [1, 10, 20, 30, 40, 50, 60]

count = 0

stim_dur = int(stim_sec / expt.frame_period())
pre = int(pre_sec / expt.frame_period())
post = int(post_sec / expt.frame_period())

T = np.arange(-pre, 2 * post + stim_dur + 1) * expt.frame_period()

for led_setting, led_df in target_df.groupby(['LED']):
    
    led_psth = np.vstack(led_df['psth'])
    
#     T = np.arange(led_psth.shape[1]) * expt.frame_period() - int(1 / expt.frame_period()) * expt.frame_period()

    mean_response = np.nanmean(led_psth, axis=0)
    sem_response = sem(led_psth, axis=0)
    
    color = colors[led_settings.index(led_setting)]
    ax.fill_between(T, mean_response - sem_response, mean_response + sem_response, color=color, alpha=0.5, lw=1)
    ax.plot(T, mean_response, color=color, label=led_setting, lw=1)

formatPlot(ax)
ax.legend()

ax.set_xlabel('Time From Stim Onset (s)', fontsize=18)
ax.set_ylabel(r'Mean Stim Response ($\sigma$)', fontsize=18)

fig.savefig('/home/sebi/plots/ind/figs3/mean_psths.svg', bbox_inches='tight')

FigureCanvasNbAgg()

In [284]:
test = df.loc[df['target']]['psth'].mean()

In [346]:
fig = plt.figure()
ax = fig.add_subplot(111)

# sns.lineplot(x='dist', y='value', hue='LED', data=merge_df)

# colors = [str(x) for x in np.linspace(0.8, 0.2, 7)]
# colors = sns.light_palette('r', 9)[-7:]
colors = sns.cubehelix_palette(7)
led_settings = [1, 10, 20, 30, 40, 50, 60]

count = 0

test_df = merge_df.loc[merge_df['dist'] <= 168.5935987978145]
max_dist = test_df['dist'].max()

for led_setting, led_df in test_df.groupby(['LED']):
    
    zeros = led_df.loc[led_df['dist'] == 0]
    rest = led_df.loc[led_df['dist'] > 0]
    values = rest['value'].values[:]
    dists = rest['dist'].values[:]
    
    means, bins, _ = binned_statistic(dists, values,
                                          statistic=np.mean,
                                          bins=np.linspace(0, max_dist, 21))

    stds, bins, _ = binned_statistic(dists, values,
                                          statistic=sem,
                                          bins=np.linspace(0, max_dist, 21))
    
    zero_mean = np.nanmean(zeros['value'])
    zero_std = sem(zeros['value'])
    
    means = np.concatenate([[zero_mean], means])
    stds = np.concatenate([[zero_std], stds])
    plot_bins = np.concatenate([[0], bins + (bins[1] - bins[0]) / 2.])
    
    ax.errorbar(plot_bins[:-1], means, yerr=0, color=colors[count], label=led_setting, fmt='o-', lw=1)
    count += 1
    
formatPlot(ax)
ax.legend()

ax.set_xlabel('Distance from Target Cell (um)', fontsize=18)
ax.set_ylabel(r'Mean Stim Response ($\sigma$)', fontsize=18)

fig.savefig('/home/sebi/plots/ind/figs3/response_vs_dist.svg', bbox_inches='tight')

FigureCanvasNbAgg()

In [43]:
traces = []
trace_stds = []

diff_means = []
diff_stds = []

roi_filter = lambda x: any([cell_name in x.tags for cell_name in ['Cell_A', 'Cell_B', 'Cell_C']])

trace_dict = {}

for expt in expts:
    
    stims = (expt.behaviorData()['led_context_pin'][:, 0] / expt.frame_period()).astype(int)
    for cell in xrange(2):
        
        
    
    out, _, xranges = ia.PSTH(ExperimentGroup([expt]), [stims[expt.totuple()]], data='raw', z_score=True,
                              pre_time=5, post_time=6, label='s2p', roi_filter=roi_filter)
    
    trace_dict[expt.totuple()] = np.array(out)

In [52]:
expt.get('tSeriesDirectory')

'/data/Mohsin/2p/msa0919_2/10_1_19/TSeriesmsa0919_2_10012019_inputoutput-003'

In [50]:
for expt in expts:
    if trace_dict[expt.totuple()].shape:
        break

In [45]:
# colors = ['0.8', '0.65', '0.5', '0.35', '0.2', '0.05']
colors = [str(x) for x in np.linspace(0.8, 0, 7)]
labels = [x.attrib['LED_setting'] for x in expts]

cell = 0

fig = plt.figure()
ax = fig.add_subplot(111)

# for trace, trace_std, color, label in zip(traces, trace_stds, colors, labels):
for expt, color, label in zip(expts, colors, labels):
    trace = trace_dict[expt.totuple()][cell, :]
    ax.plot(xranges[0], trace, color=color, label=label)
#     ax.fill_between(x=xranges[0], y1=trace - trace_std, y2=trace + trace_std, color=color, alpha=0.2)
    
# ax.axvspan(0, 1, alpha=0.2, color='red')

sns.despine()
ax.legend()

ax.set_xlabel('Time From Laser Onset (s)', fontsize=10)
ax.set_ylabel('Raw Fluorescence (Z-Score)', fontsize=10)

ax.set_title('Targeted Stimulation Response', fontsize=10);

# fig.savefig('/home/sebi/plots/ind/laser_io_psth_cell_{}.pdf'.format(cell))

FigureCanvasNbAgg()

IndexError: too many indices for array

In [46]:
fig = plt.figure()
ax = fig.add_subplot(111)

sec = int(1 / expt.frame_period())
zero = 5 * sec
window = 2 * sec

for color, cell in zip(['0.8', 'r', '0.2'], xrange(3)):
    
    for expt, label in zip(expts, labels):
        trace = trace_dict[expt.totuple()][cell, :]
        diff = np.mean(trace[zero:zero + window]) - np.mean(trace[zero - window:zero])
        
        if label == 1:
            cell_label = 'cell {}'.format(cell)
        else:
            cell_label = None
        ax.plot(label, diff, '.', color=color, ms=12, label=cell_label)

sns.despine()
ax.set_xlabel('Laser Power (mW)', fontsize=10)
ax.set_ylabel('Post - Pre \n(2s, Raw z-score)', fontsize=10)

ax.set_title('Response Curves', fontsize=10)

ax.legend()

# fig.savefig('/home/sebi/plots/ind/laser_io_response_curves.pdf')

FigureCanvasNbAgg()

IndexError: too many indices for array

In [181]:
fig = plt.figure()
ax = fig.add_subplot(111)

sec = int(1 / expt.frame_period())
zero = 5 * sec
window = 2 * sec

cells = np.array([1, 2])

# for color, cell in zip(['0.8', 'r', '0.2'], xrange(3)):
cell_means = []
cell_stds = []

for expt, label in zip(expts, labels):
    
    trace = trace_dict[expt.totuple()][cells, :]

    diffs = np.mean(trace[:, zero:zero + window], axis=1) - np.mean(trace[:, zero - window:zero], axis=1)

    cell_means.append(np.mean(diffs))
    cell_stds.append(np.std(diffs))
      
ax.errorbar(labels, cell_means, yerr=cell_stds, color='k', marker='.',
           ms=8, capsize=3, capthick=1.5, ls='none')

sns.despine()
ax.set_xlabel('Laser Power (mW)', fontsize=10)
ax.set_ylabel('Post - Pre \n(2s, Raw z-score)', fontsize=10)

# ax.set_title('Response Curve', fontsize=10)

fig.savefig('/home/sebi/plots/ind/laser_io_agg_response_curve.pdf')

FigureCanvasNbAgg()